# Process wifi dataset
Consider t1, t2, t3 three domains

In [33]:
from scipy.io import loadmat
from scipy.io import savemat
from os.path import join
import numpy as np
np.set_printoptions(linewidth=120)

Create .npz data by random subsampling 10 subsets

In [42]:
# extract data
target_id = 3
num_train = 700
num_domain = 3
data_path = join('../data/wifi/', 'aligned_data.mat')
data_wifi = loadmat(data_path)
x_t1 = data_wifi['X1_new']
y_t1 = data_wifi['Y1_new']
x_t2 = data_wifi['X2_new']
y_t2 = data_wifi['Y2_new']
x_t3 = data_wifi['X3_new']
y_t3 = data_wifi['Y3_new']
ss1 = x_t1.shape[0]
ss2 = x_t2.shape[0]
ss3 = x_t3.shape[0]
x = np.concatenate((x_t1, x_t2, x_t3), 0)
y = np.concatenate((y_t1, y_t2, y_t3), 0)
if target_id == 3:
    label_d = np.concatenate((np.ones((ss1, 1))*0, np.ones((ss2, 1))*1, np.ones((ss3, 1))*2))
elif target_id == 2:
    label_d = np.concatenate((np.ones((ss1, 1))*0, np.ones((ss2, 1))*2, np.ones((ss3, 1))*1))
else:
    label_d = np.concatenate((np.ones((ss1, 1))*2, np.ones((ss2, 1))*0, np.ones((ss3, 1))*1))

y = np.concatenate((y, label_d), 1)

# extract Markov Blanket
# MB = dag_mat['MB'].flatten() - 1
# x = x[:, MB[:-2]]

# random subsamspling
for seed in range(1, 11):
    full_path = join('../data/wifi/', 'tot%d_dataId%d.npz'%(target_id, seed))
    np.random.seed(seed)
    total_num_samples = y.shape[0]
    indices = np.arange(total_num_samples)
    np.random.shuffle(indices)
    x_sub = x[indices[0:num_train*num_domain]]
    y_sub = y[indices[0:num_train*num_domain]]
    # save data
    np.savez(full_path, x=x_sub, y=y_sub)

Convert .npz to .mat format

In [10]:
for dataId in range(1, 11):
    for targetId in range(1, 4):
        file_path = '../data/wifi/tot%d_dataId%d.npz' % (targetId, dataId)
        file_path_mat = '../data/wifi/tot%d_dataId%d.mat' % (targetId, dataId)
        npzfile = np.load(file_path)
        list(npzfile.keys())
        savemat(file_path_mat, mdict={'x':npzfile['x'],'y':npzfile['y']})

Create python data_mat and Markov Blanket

In [45]:
dag_mat_file = '../data/wifi/t1t3_dag.mat'
dag_wifi = loadmat(dag_mat_file)
dag_mat = dag_wifi['gn']
MB = dag_wifi['MB'].flatten() - 1
dag_mat = dag_mat.T
# extract MB in the DAG if using MB data
dag_mat = dag_mat[np.ix_(MB[:-2], MB)]
print(dag_mat.shape)
full_path = '../data/wifi/t1t3_dag.npz'
np.savez(full_path, mat=dag_mat, MB=MB)

(16, 18)


Look at the data

In [12]:
data_path = join('../data/wifi/', 'aligned_data.mat')
data = loadmat(data_path)
y = data['Y1_new']
unique_y = np.unique(y)
print(unique_y.shape)

(19,)


In [16]:
data_path = join('../data/wifi/', 't1t2_dag.mat')
data = loadmat(data_path)
MB = data['MB'].flatten()-1
print(MB)
dag_mat = data['gn'].T
# data['gn'][np.ix_(MB, MB)].T
dag_mat[np.ix_(MB[:-2], MB)].shape 

[ 0  1  2  3  4  5  6  7 10 11 12 13 14 43 15 19 67 68]


(16, 18)

In [14]:
data_path = join('../data/wifi/', 't2t3_dag.mat')
data = loadmat(data_path)
data['MB'].shape

(1, 19)

In [15]:
data_path = join('../data/wifi/', 't1t3_dag.mat')
data = loadmat(data_path)
data['MB'].shape

(1, 18)

In [17]:
npzfile = np.load('../data/wifi/tot1_dataId1.npz')
print(npzfile['x'].shape)
print(npzfile['y'].shape)

(2100, 17)
(2100, 2)
